In [ ]:
import sys
print(sys.executable)


In [ ]:
!{sys.executable} -m pip install selenium

In [ ]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin

##### Web scrapper for infinite scrolling page #####
driver = webdriver.Chrome()
driver.get("https://www.espncricinfo.com/series/australia-tour-of-india-2013-14-647237/india-vs-australia-2nd-odi-647251/ball-by-ball-commentary")
time.sleep(5)  # Allow 2 seconds for the web page to open
scroll_pause_time = 3 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break 

##### Extract Reddit URLs #####
ball_numbers=list()
soup = BeautifulSoup(driver.page_source, "html.parser")


In [ ]:
ball_number=list()

In [ ]:
for div in soup.findAll('span', {'class':"ds-text-tight-s ds-font-regular ds-mb-1 lg:ds-mb-0 lg:ds-mr-3 ds-block ds-text-center ds-text-typo-mid1"}):
    ball_number.append(div.text)

In [ ]:
ball_number

In [ ]:
x=list()

In [ ]:
for div in soup.findAll('div', {'class':"ds-text-tight-m ds-font-regular ds-flex ds-px-3 ds-py-2 lg:ds-px-4 lg:ds-py-[10px] ds-items-start ds-select-none lg:ds-select-auto"}):
    x.append(div.text)

In [ ]:
x

In [ ]:
import re
import pandas as pd

# Given list of strings

# Define regular expressions for each category
over_ball_pattern = re.compile(r'(\d+\.\d)([\d•Wnblbb]+)(\w+) to (\w+(\s\w+)?)(,\s(.+))?')

# Extract information using regular expressions
data = []
for ball in x:
    match = over_ball_pattern.search(ball)
    if match:
        over_ball, runs_str, bowler, batsman, _ ,_,batman= match.groups()
        if runs_str.isdigit():
            runs = int(runs_str)
        elif runs_str == '•':
            runs = 0
        elif runs_str == 'W':
            runs= "Out"
        elif runs_str[:-2].isdigit() and runs_str.endswith("nb"):
            runs = runs_str
        elif runs_str[:-2].isdigit() and runs_str.endswith("lb"):
            runs = runs_str
        elif runs_str[:-1].isdigit() and runs_str.endswith("b"):
            runs = runs_str
        else:
            # Handle other cases if needed
            runs = None

        data.append({
            'Over_Ball': over_ball,
            'Bowler': bowler,
            'Batsman': batsman,
            'Runs': runs,
            'Commentary':_
        })

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

In [ ]:
df.tail(10)

In [ ]:
df['Commentary'].head(5)

In [ ]:
# Function to classify side based on commentary
def classify_side(commentary):

    fine_leg_keywords = ["fine-leg", "fine leg", "backward short leg", "backward-short-leg", "deep fine leg", "long leg", "leg gully", "deep fine-leg"]
    third_man_keywords = ["third man", "third-man", "deep third", "deep third man", "short third man", "short third-man", " slip", "slips"]
    point_keywords = ["point", "backward point", "backward-point", "silly point", "silly-point", "deep cover point", "deep backward point", "deep backward"]
    cover_keywords = ["cover", "extra cover", "extra-cover", "deep cover", "deep extra cover"]
    mid_off_keywords = ["mid-off", "mid off", "deep mid-off", "deep mid off", "long-off", "long off", "straight mid off", "off"]
    mid_on_keywords = ["mid-on", "mid on", "deep mid-on", "deep mid on", "long on", "long-on", "deep long on", "deep long-on", "on"]
    mid_wicket_keywords = ["mid-wicket", "mid wicket", "deep mid-wicket", "deep mid wicket", "midwicket"]
    square_leg_keywords = ["square leg", "square-leg", "deep square leg", "deep square-leg"]
    out_keywords = [ "bowled"]
    no_ball = ["no-ball"]

    # Normalize the commentary text
    commentary_lower = commentary.lower()

    # Check each fielding position
    if any(keyword in commentary_lower for keyword in fine_leg_keywords):
        return "fine leg"
    elif any(keyword in commentary_lower for keyword in third_man_keywords):
        return "third man"
    elif any(keyword in commentary_lower for keyword in point_keywords):
        return "point"
    elif any(keyword in commentary_lower for keyword in cover_keywords):
        return "cover"
    elif any(keyword in commentary_lower for keyword in mid_wicket_keywords):
        return "mid wicket"
    elif any(keyword in commentary_lower for keyword in square_leg_keywords):
        return "square leg"
    elif any(keyword in commentary_lower for keyword in mid_off_keywords):
        return "mid off"
    elif any(keyword in commentary_lower for keyword in mid_on_keywords):
        return "mid on"
    elif any(keyword in commentary_lower for keyword in out_keywords):
        return "out"
    elif any(keyword in commentary_lower for keyword in no_ball):
        return "no_ball"
    
    
    
   
    
    return "unknown position"

# Apply the function to classify side and add a new column
df['Side'] = df['Commentary'].apply(classify_side)

In [ ]:
df['Side'].head

In [ ]:
unknown_count = (df['Side'] == "unknown position").sum()
print("Number of unknown positions:", unknown_count)

In [ ]:
file_name = 'cricket_scores100.xlsx'
df.to_excel(file_name, index=False)